In [1]:
## Imports
import numpy as np
import pandas as pd
from sklearn.preprocessing import normalize
from ast import literal_eval

In [2]:
DATASET_NAME = 'DI2KG'

In [3]:
class LSH :
  
    # Random limits, const
    __max_rand =  1000
    __min_rand = -1000
    
    # Constructor
    def __init__(self, k, L, embedding_size = 150):
        
        # Number of hash function
        self.k = k
        
        # Number of attempts
        self.L = L
        
        # Embedding length 
        self.embedding_size = embedding_size
        
        # Random matrices
        self.normalized_random_matrices = []
        
        for i in range(self.L):
            random_matrix = np.random.randint(self.__min_rand, self.__max_rand,(self.k, self.embedding_size));
            
            # Append normalized random matrices
            self.normalized_random_matrices.append(normalize(random_matrix, axis=1, norm='l1'))
        
    
    # Locality Sensitive hash function
    def locality_sensitive_hash(self, embedding, matrix_index):
        out = 0
      
        for h in self.normalized_random_matrices[matrix_index]:
            if (np.dot(h, embedding) >= 0):
                out = (out << 1) | 1
            else:
                out = (out << 1) | 0

        return out
      
    # Divide in buckets using L-th matrix
    def divide_in_buckets(self, embeddings, matrix_index):
        out = {}
        for embedding in embeddings:
            hash = self.locality_sensitive_hash(embedding, matrix_index)
            if (hash in out):
                out[hash].append(embedding)
            else:
                out[hash] = [embedding]
            
        return out  


In [4]:
test = LSH(k=5, L=2, embedding_size=10)
print(test.normalized_random_matrices)

print(test.locality_sensitive_hash([1,2,3,4,5,6,7,8,9,10], 1))

[array([[ 0.02352312, -0.00855386, -0.13766373, -0.14595028, -0.01095964,
        -0.19326383,  0.03448276, -0.23175621,  0.20128308, -0.01256349],
       [-0.17351916, -0.04390244,  0.1029036 , -0.18885017, -0.03182346,
         0.01277584,  0.16445993, -0.01068525,  0.11056911, -0.16051103],
       [ 0.09033424, -0.1321891 ,  0.12767239,  0.12104788,  0.17464619,
        -0.09003312, -0.00542005, -0.17856067, -0.06654622, -0.01355014],
       [ 0.18655266, -0.13797124,  0.02545667,  0.08608628, -0.0551885 ,
        -0.11562379, -0.05071901, -0.1467159 , -0.17567042, -0.02001555],
       [-0.16437537, -0.00079697,  0.06056983, -0.18748755, -0.16935645,
         0.07989639, -0.0938434 ,  0.04562662,  0.13369197, -0.06435545]]), array([[ 0.11229947,  0.18015858, -0.03651116,  0.10842707, -0.06214273,
         0.12686705,  0.13682464,  0.01567398,  0.12520745, -0.09588788],
       [-0.23640167,  0.08403068,  0.0292887 , -0.05648536,  0.02475593,
         0.12726639, -0.05788006,  0.20118

In [5]:
## almost equals
embeddings = [[1.2345,2,3,4,5,6,10.4,8,9,10],[1,2,3,4,5,6,7,8,9,10],[1,2,3,5,5,6,7,8,9,10]]
print(len(test.divide_in_buckets(embeddings, 1)))

## not equals
embeddings = np.random.randint(-10000, 10000,(10000, 10))
print(len(test.divide_in_buckets(embeddings, 1)))


3
32


In [6]:
## TEST BLOCKING PERFORMANCE
#    Basta che per ogni tupla vado a prendere la sua corrispondente, ne calcolo
#     i vari L hash e controllo che almeno uno sia uguale e incremento un
#     contatore. La precisione è contatore/numero di tuple controllate, giusto?
def performance_test(filtered_dataset, k, L, embedding_size):
    
    match_found = 0
    
    lsh = LSH(k, L, embedding_size)
    
    # for each elemt in dataset
    for index, row in filtered_dataset.iterrows():
        x_embedding = np.array(literal_eval(row['left_table']))
        y_embedding = np.array(literal_eval(row['right_table']))
          
        x_hashs = set()
        y_hashs = set()
        for i in range(L):
            x_hashs.add(lsh.locality_sensitive_hash(x_embedding, i))
            y_hashs.add(lsh.locality_sensitive_hash(y_embedding, i))
        
        if (len(set.intersection(x_hashs, y_hashs)) > 0):
            match_found += 1
  
    
    return match_found / len(filtered_dataset.index)

In [7]:
## Open dataset 
df = pd.read_csv('../../lsh-test-data/' + DATASET_NAME + '-embeddings.csv')

## Remove 0 labled
df = df[df.label == 1]

precision_max = 0
k_max = 0
L_max = 0
for k in range(30):
    for L in range(10):
        precision = performance_test(df, k + 1, L + 1, 150)
        print("K: {0}, L: {1}, Precision:{2}".format(k + 1, L + 1, precision))
        if (precision >= precision_max):
            precision_max = precision
            k_max = k + 1
            L_max = L + 1

print("Max precision: {0}, k: {1}, L: {2}".format(precision_max, k_max, L_max))

K: 1, L: 1, Precision:1.0
K: 1, L: 2, Precision:1.0
K: 1, L: 3, Precision:1.0
K: 1, L: 4, Precision:1.0
K: 1, L: 5, Precision:1.0
K: 1, L: 6, Precision:1.0
K: 1, L: 7, Precision:1.0
K: 1, L: 8, Precision:1.0
K: 1, L: 9, Precision:1.0
K: 1, L: 10, Precision:1.0
K: 2, L: 1, Precision:0.6086956521739131
K: 2, L: 2, Precision:0.782608695652174
K: 2, L: 3, Precision:1.0
K: 2, L: 4, Precision:1.0
K: 2, L: 5, Precision:1.0
K: 2, L: 6, Precision:1.0
K: 2, L: 7, Precision:1.0
K: 2, L: 8, Precision:1.0
K: 2, L: 9, Precision:1.0
K: 2, L: 10, Precision:1.0
K: 3, L: 1, Precision:0.6086956521739131
K: 3, L: 2, Precision:0.6521739130434783
K: 3, L: 3, Precision:1.0
K: 3, L: 4, Precision:0.8695652173913043
K: 3, L: 5, Precision:1.0
K: 3, L: 6, Precision:1.0
K: 3, L: 7, Precision:1.0
K: 3, L: 8, Precision:1.0
K: 3, L: 9, Precision:1.0
K: 3, L: 10, Precision:1.0
K: 4, L: 1, Precision:0.5217391304347826
K: 4, L: 2, Precision:0.6956521739130435
K: 4, L: 3, Precision:0.782608695652174
K: 4, L: 4, Precision

K: 21, L: 8, Precision:0.13043478260869565
K: 21, L: 9, Precision:0.21739130434782608
K: 21, L: 10, Precision:0.2608695652173913
K: 22, L: 1, Precision:0.0
K: 22, L: 2, Precision:0.17391304347826086
K: 22, L: 3, Precision:0.13043478260869565
K: 22, L: 4, Precision:0.13043478260869565
K: 22, L: 5, Precision:0.21739130434782608
K: 22, L: 6, Precision:0.08695652173913043
K: 22, L: 7, Precision:0.17391304347826086
K: 22, L: 8, Precision:0.13043478260869565
K: 22, L: 9, Precision:0.2608695652173913
K: 22, L: 10, Precision:0.21739130434782608
K: 23, L: 1, Precision:0.043478260869565216
K: 23, L: 2, Precision:0.21739130434782608
K: 23, L: 3, Precision:0.043478260869565216
K: 23, L: 4, Precision:0.08695652173913043
K: 23, L: 5, Precision:0.21739130434782608
K: 23, L: 6, Precision:0.043478260869565216
K: 23, L: 7, Precision:0.13043478260869565
K: 23, L: 8, Precision:0.21739130434782608
K: 23, L: 9, Precision:0.17391304347826086
K: 23, L: 10, Precision:0.13043478260869565
K: 24, L: 1, Precision: